# MP 4: Robot Motion Control

**Due date**: Feb 27, 2021 at 10:45am.

**Instructions**: Read and complete the problems below. In this assignment, you should be switched over to a local install.  As a backup, you may run code through this notebook via Binder, but be aware that you will be editing external .py files, and saving your progress is not as simple as clicking "Save notebook state" in this window.

To submit your assignment, perform the following:

1. Double-check that your programs run without error.
2. Send this file, all of your .py files, .npy files, and any other files you used in your programs on Moodle [http:/learn.illinois.edu](http:/learn.illinois.edu).
3. If you are using any external libraries other than the ones that are indicated during the installation process, include a README file indicating which library and version you are using.  To be on the safe side, you should include a backup procedure until the graders verify that they are able to support use of that library.

## Problem 1: Reachable workspace calculation

Given a robot and an end effector point, the reachable workspace is the set of points that the end effector can attain while respecting various constraints.  These constraints include joint limits, self-collisions, and environmental collisions.  In this problem we will approximate the reachable workspace using a volumetric grid.  

There are in general two algorithms for calculating a reachable workspace.  The first samples the configuration space, and then determines the resulting world coordinates of the end effector.  The second iterates along cells of the grid, and then tries to find a feasible configuration that matches the end effector to the grid cell using IK.  We'll explore both of these in this question.

Your code for this problem will go inside `problem1.py`.  Don't forget to check for self collisions and collisions with the obstacles list.

### Problem 1.A

Implement the `calculate_workspace_free` method to calculate the reachable workspace of the given end effector point. The grid is constructed for you as a Numpy array with resolution 20x20x20. If you have not used Numpy in the past, its basic usage is shown in the example code, which simply marks the cell in which the current end effector point is located.  Adapt this code to approximate the reachable workspace by sampling from the set valid configurations.  Note that your result will automatically saved to `problem1a.npy` as well as displayed on screen.  Use 10,000 configuration samples for debugging, and then try sampling 100,000 configuration samples for your final result.

### Problem 1.B

Implement the `calculate_workspace_axis` method, which constrains that the "pen" axis points along a specified work-space axis.  In this function, you will loop through a 15x15x15 numpy array, create the IK constraint matching the pen tip to the cell center and the pen axis to the designated axis, and then solve for a IK solution.

Because IK can get caught in local minima, you may want to use a random restart technique.  The `ik.solve_global` function may be helpful here, but beware that its default parameters use many IK solver iterations and many restarts; you may wish to set them lower for efficiency's sake.

Test this from the command line by uncommenting `problem_1b()` at the end of the file.

### Problem 1.C

The `problem_1c` function will compute multiple reachable workspaces, for the pen pointing up, down, and sideways.  Create a new visualization function so that you can show the *difference* between each workspace; i.e., the 6 possible differences `Wup - Wdown`, `Wdown - Wup`, `Wup - Wside`, `Wside - Wup`, `Wdown - Wside`, and `Wside - Wdown`.  (The differences can be computed with just a few Numpy calls).  Qualitatively, describe the differences between each pair of workspaces in the space marked below.

## Problem 2: Cartesian motion control

This is the first problem in which you will interact with a control loop for a "real" robot, rather than playing with models.  In this problem the robot is carrying a pen, and the job of the controller to rotate the pen around a vertical axis, creating a circle.  Specifically, the pen tip *and its axis* should revolve around a given vertical axis (in world space), offset from the pen tip by a given radius.

### Problem 2.A

Fill out the `CircleController` class, specifically the `advance` method, to execute this circle motion.  You will know when you are done when the robot's motion is smooth, and the trace displayed is very close to a perfect circle.

The `advance` method will be called repeatedly to drive the robot forward around the circle by solving several incremental inverse kinematics problems.  You will be interacting with `self.controller`, which is an interface to an "external" robot controller that conforms to the [Robot Interface Layer](http://motion.cs.illinois.edu/software/klampt/latest/pyklampt_docs/Manual-Control.html#using-the-ril-api) guidelines.

Writing a control loop is very different from writing procedural code.  Your loop operates by reading from the robot controller and feeding it incremental position commands, and only does a small amount of work every time `advance()` is called.  It is your loop's job to maintain any internal state that you'd like to persist and change between `advance()` calls, e.g., progress along the current task.  The robot controller can only accept one type of command per advance call, so if you try to set multiple position commands, only the last one will actually be accepted.

Note that it can be difficult to debug fast control loops.  On a real robot, logging is typically the only way you can piece together what could cause your system to go wrong, but in this simulation, you can go ahead and use `print(...)` and `input()` to step through your loop.

There are a couple of quirks with the robot controller interface that should be noted.  First, it operates only on the robot's motors, and the number of motors is less than the URDF model's degrees of freedom, because the URDF model has some "phantom" links that are just fixed to one another.  (This need for a translation layer is actually very typical for industrial robots; just be glad we're not asking you to convert to/from joint encoder ticks!)  To convert a configuration to the robot controller, use `controller.configFromKlampt(qklampt)`, and to convert a velocity to the robot controller, use `controller.velocityFromKlampt(vklampt)`.  To do the reverse, use `controller.configToKlampt(qcontroller)` and `controller.velocityToKlampt(vcontroller)`. 

Second, although the Robot Interface Layer has many options, in this problem you should consider the following four possible ways of commanding the robot's motion:
- `controller.setPosition(q)` sends an *immediate* position command that will be reflected instantaneously.
- `controller.setVelocity(v,ttl=None)` sends an immediate velocity command that will continue moving the robot for some time, even if no other commands are sent.
- `controller.setPID(q,dq,torque=None)` sends a Position-Integral-Derivative (PID) command, which generates motor torques according to a fast feedback controller.
- `controller.setPiecewiseLinear(durations,milestones)` sends a linearly-interpolated set of milestones, spaced in time by `durations` seconds.  If you send a long-duration motion, you can use the `controller.isMoving()` method to check whether the controller is done.

Third, the robot has four continuous rotation joints, and IK solver can have the joints can go amazingly out of the range 0 to 2pi.  The controller doesn't know anything about this, so `setPiecewiseLinear` in particular can make the robot go crazy.  Passing the (Klampt) configuration through the `normalize_config` function via `q = normalize_config(q,qcur)` after solving IK will avoid this issue.

Finally, if the simulated robot self collides or exits its joint range, it will go into an error status, and you will lose control.  Your final code should keep the robot in a nominal ('ok') status.

### Problem 2.B

In `run_simulation`, by default the simulation is a "kinematic simulation" that doesn't consider physics at all; it just feeds back what commands you send to the robot back to you.  There is a commented out line instantiating a `SimPositionControlInterface`, which instantiates a controller that would actually run a physics simulation, including gravity, motor forces, etc.  Uncomment this line and run your simulation again.  Is the trace as precise?  

Now, try editing the URDF file `kinova_gen3_7dof.urdf` to change how the simulator emulates the robot's behavior.  At the bottom, you see several lines of the form `<link .../>`.  Create a backup of this file, and reduce the `servoP` values by a factor of 10. Now observe the simulation trace.  What happened?
    
(Don't include the modified URDF file in your submission. In fact you should revert to your backup after you are done with this question.)
  
In the space below for written answers, explain at least three reasons why a simulated robot may not behave like a planned motion.  Also, explain at least 2 reasons why a simulated robot may not behave like a real robot.

## Problem 3: State machines and drawing

State machines are used when performing sequential, disparate functions inside a control loop.  In this problem, you will implement a state machine to implement the behavior of a drawing robot. 

### Problem 3.A

In the `get_paths` function, design a sequence of strokes (2D paths) that trace out your initials.  If you install the `svgpathtools` package, you can import these paths from an SVG file.  (Bezier curves are supported, so get fancy if you wish!)  The result will be a list of Trajectory objects.

In the `DrawingController` class, you will be given your strokes translated to a plane that's within your robot's reach.  In its `advance()` method, implement a state machine to "draw" these strokes.  The logic of the state machine should be to move to each stroke's start, lower down the pen in a controlled fashion, trace the path, lift up the pen, and then proceed to the next stroke.  At the end, it should return to the home configuration `qhome`. 

As indicated in the constructor, you will have to maintain some internal state in `self`.  Here, you may add whatever variables you'd like to implement the drawing logic, including intermediate targets, paths, etc.

When using the `controller.setPiecewiseLinear` function, e.g., when moving to each stroke's start and moving home, you may find it convenient to define "waiting" states that proceed to the next state only when the robot is done moving.  You may also find it convenient to plan a Cartesian path, store it, and then switch to a "path execution" state that walks along the path.
    
*Note: this problem involves substantial implementation work*.  Don't wait until the last minute to start on it!  My implementation is 70 very dense lines of code, but took several rounds of debugging to get exactly right.

### Problem 3.B (IR2 section only)

Investigate what happens with your controller if the table scale or offset in `main()` is placed such that the strokes are out of reach of the robot.  Implement an error-checking functionality such that the robot skips to the next stroke if it can't solve an IK problem.

## Written responses

### Written response for Problem 1.C

Put your answer here.

### Written response for Problem 2.B

Put your answer here.

## Jupyter cells for non-local installs

In [ ]:
%load_ext autoreload
%autoreload 1
from klampt import vis

In [ ]:
## Problem 1: Reachable workspace calculation
%aimport problem1

vis.createWindow()
problem1.problem_1a()
#call this to re-show a prior saved reachable workspace
#problem1.show_problem_1a()

In [ ]:
%aimport problem1

vis.createWindow()
problem1.problem_1b()
#call this to re-show a prior saved reachable workspace
problem1.show_problem_1b()

In [ ]:
%aimport problem1

vis.createWindow()
#problem1.problem_1c()
#call this to re-show a prior saved reachable workspace
problem1.show_problem_1c()

In [ ]:
%aimport problem2

vis.createWindow()
problem2.main()


In [ ]:
%aimport problem3

vis.createWindow()
problem3.main()